In [20]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, unix_timestamp, max

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/02 15:09:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark.version

'3.3.2'

In [6]:
df = spark.read.option("header", True).parquet("yellow_tripdata_2024-10.parquet")

In [7]:
df_repartitioned = df.repartition(4)

In [8]:
df_repartitioned.write.mode("overwrite").parquet("homework_W5/")

In [9]:
!du -sh homework_W5/*.parquet

25M	homework_W5/part-00000-d508ccc3-a4b7-47af-bf6d-022e2399a3d7-c000.snappy.parquet
25M	homework_W5/part-00001-d508ccc3-a4b7-47af-bf6d-022e2399a3d7-c000.snappy.parquet
25M	homework_W5/part-00002-d508ccc3-a4b7-47af-bf6d-022e2399a3d7-c000.snappy.parquet
25M	homework_W5/part-00003-d508ccc3-a4b7-47af-bf6d-022e2399a3d7-c000.snappy.parquet


In [12]:
df_filtered = df.withColumn("pickup_date", to_date(col("tpep_pickup_datetime"))) \
                .filter(col("pickup_date") == "2024-10-15")

# Count the number of trips
trip_count = df_filtered.count()
print(f"Number of taxi trips on October 15, 2024: {trip_count}")

Number of taxi trips on October 15, 2024: 128893


In [13]:
df

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp, tpep_dropoff_datetime: timestamp, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double]

In [14]:
df.head()

Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 10, 1, 0, 30, 44), tpep_dropoff_datetime=datetime.datetime(2024, 10, 1, 0, 48, 26), passenger_count=1, trip_distance=3.0, RatecodeID=1, store_and_fwd_flag='N', PULocationID=162, DOLocationID=246, payment_type=1, fare_amount=18.4, extra=1.0, mta_tax=0.5, tip_amount=1.5, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=24.9, congestion_surcharge=2.5, Airport_fee=0.0)

In [21]:
df_with_duration = df.withColumn(
    "trip_duration_hours", 
    (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600
)

In [22]:
longest_trip = df_with_duration.select(max("trip_duration_hours")).collect()[0][0]

print(f"Durata del viaggio più lungo: {longest_trip:.2f} ore")

Durata del viaggio più lungo: 162.62 ore


In [23]:
# Load the taxi zone lookup CSV
zone_df = spark.read.option("header", True).option("inferSchema", True).csv("taxi_zone_lookup.csv")

# Create a temporary view
zone_df.createOrReplaceTempView("zone_lookup")

In [25]:
# Load the Yellow Taxi October 2024 dataset (update path accordingly)
taxi_df = spark.read.option("header", True).option("inferSchema", True).parquet("yellow_tripdata_2024-10.parquet")

# Create a temporary view
taxi_df.createOrReplaceTempView("yellow_taxi")

In [26]:
least_frequent_zone = spark.sql("""
    SELECT 
        z.Zone AS pickup_zone,
        COUNT(t.PULocationID) AS trip_count
    FROM yellow_taxi t
    JOIN zone_lookup z 
    ON t.PULocationID = z.LocationID
    GROUP BY z.Zone
    ORDER BY trip_count ASC
    LIMIT 1
""")

least_frequent_zone.show()


+--------------------+----------+
|         pickup_zone|trip_count|
+--------------------+----------+
|Governor's Island...|         1|
+--------------------+----------+

